In [1]:
import numpy as np
import scipy.linalg as la
import math

In [2]:
def solve_DARE(A, B, Q, R):
    """
    solve a discrete time_Algebraic Riccati equation (DARE)
    """
    X = Q
    Xn = Q
    max_iter = 150
    eps = 0.01

    for i in range(max_iter):
        Xn = A.T @ X @ A - A.T @ X @ B @ \
            la.inv(R + B.T @ X @ B) @ B.T @ X @ A + Q
        if (abs(Xn - X)).max() < eps:
            break
        X = Xn

    return Xn


def dlqr(A, B, Q, R):
    """Solve the discrete time lqr controller.
    x[k+1] = A x[k] + B u[k]
    cost = sum x[k].T*Q*x[k] + u[k].T*R*u[k]
    # ref Bertsekas, p.151
    """

    # first, try to solve the ricatti equation
    X = solve_DARE(A, B, Q, R)

    print("X: ", X)

    # compute the LQR gain
    K = la.inv(B.T @ X @ B + R) @ (B.T @ X @ A)

    eigVals, eigVecs = la.eig(A - B @ K)

    return K, X, eigVals

In [3]:
v = 3.8
L = 2.5
dt = 0.1

e = 0.1
pe = 0.11
th_e = 0.1
pth_e = 0.11

# LQR parameter
Q = np.eye(4)
R = np.eye(1)

In [5]:
A = np.zeros((4, 4))
A[0, 0] = 1.0
A[0, 1] = dt
A[1, 2] = v
A[2, 2] = 1.0
A[2, 3] = dt
# print(A)

B = np.zeros((4, 1))
B[3, 0] = v / L

K, _, _ = dlqr(A, B, Q, R)

x = np.zeros((4, 1))

x[0, 0] = e
x[1, 0] = (e - pe) / dt
x[2, 0] = th_e
x[3, 0] = (th_e - pth_e) / dt

print(K)
(-K @ x)


X:  [[ 16.07489781   1.50748978  20.04334124   1.46948801]
 [  1.50748978   1.15074898   2.00433412   0.1469488 ]
 [ 20.04334124   2.00433412 101.78053648   7.87240668]
 [  1.46948801   0.1469488    7.87240668   1.73140012]]
[[0.44670409 0.04467041 2.56285061 0.23931031]]


array([[-0.2725574]])